In [25]:
import pandas as pd
try:
    import pandopt as pdo
except:
    import sys, os
    sys.path.append('/'.join(os.getcwd().split('/')[:-2]))
    import pandopt as pdo
import pandas as pd
import numpy as np
import tqdm 
import pandas as pd
import numpy as np
import timeit
import functools

import ast
import inspect
import types
import numpy as np
import numba as nb
import pandas as pd
import time
import logging
import functools
import copy
import sys
from typing import Callable, Type, Dict, Tuple, Any
from numpy.lib.stride_tricks import sliding_window_view
logger = logging.getLogger()
logger.setLevel(0)




class DF(pd.DataFrame):
    def groupby(self, *args, **kwargs):
        print(args, kwargs)
        return DGRoup(self,*args, **kwargs)


class DGRoup(pd.core.groupby.generic.DataFrameGroupBy):
    def __init__(self, *args, **kwargs):
        print('init:',args, kwargs)
        super().__init__(*args, **kwargs)

    def apply(self, *args, **kwargs):
        print('apply', args, kwargs)
        result = super().apply(*args, **kwargs)
        print('result', result)
        return result

df = pdo.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D'])#.astype(np.float32)
df['E'] = df['A'].apply(lambda x: 1 if x <-2 else 2 if x <-1 else 3 if x<0 else 4 if x<2 else 5)
df.dtypes

A


A    float64
B    float64
C    float64
D    float64
E      int64
dtype: object

In [1]:
import pandopt as pdo
import pandas as pd
import numpy as np
dfa=pd.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D'])
dfa['E'] = dfa['A'].apply(lambda x: 1 if x <-2 else 2 if x <-1 else 3 if x<0 else 4 if x<2 else 5)
dfa.loc[dfa.groupby('E').groups[1],:]

             A         B         C         D  E
68   -2.024255 -0.653233  0.787701 -1.877896  1
100  -2.165115 -0.510235 -0.905067  0.326226  1
102  -2.353347 -1.576216  0.298123 -0.868118  1
111  -2.162830  2.608022  0.129211  0.355969  1
114  -2.386508  0.178014  0.548628 -1.234875  1
...        ...       ...       ...       ... ..
9644 -2.287872  0.477173 -1.245266  0.587293  1
9697 -2.997249 -0.876881 -2.087745 -0.105303  1
9791 -2.012931 -0.633578 -2.998835  0.023668  1
9807 -2.151815  0.838173 -1.599651 -0.020868  1
9910 -2.028642 -0.801516 -1.350986 -0.010207  1

[215 rows x 5 columns]

In [7]:
df = pdo.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D'])#.astype(np.float32)
df['E'] = df['A'].apply(lambda x: 1 if x <-2 else 2 if x <-1 else 3 if x<0 else 4 if x<2 else 5)
groups = df.groupby('E').groups

A
E


{1: [93, 127, 134, 154, 170, 173, 236, 237, 238, 331, 365, 392, 434, 453, 497, 601, 629, 671, 708, 747, 785, 799, 853, 1034, 1035, 1111, 1123, 1141, 1283, 1458, 1475, 1509, 1587, 1633, 1651, 1666, 1726, 1751, 1764, 1823, 1826, 1939, 1962, 2023, 2035, 2077, 2141, 2150, 2194, 2238, 2245, 2274, 2328, 2348, 2425, 2552, 2595, 2628, 2648, 2684, 2789, 2829, 2838, 2839, 2845, 2957, 2961, 3038, 3085, 3088, 3097, 3100, 3146, 3149, 3170, 3193, 3243, 3264, 3267, 3330, 3487, 3509, 3520, 3570, 3702, 3741, 3744, 3747, 3773, 3781, 3795, 3837, 3843, 3928, 3955, 3960, 3980, 3998, 4102, 4177, ...], 2: [5, 24, 38, 39, 64, 67, 76, 80, 88, 95, 98, 99, 105, 110, 123, 124, 135, 139, 142, 151, 157, 180, 182, 184, 191, 200, 202, 204, 212, 224, 233, 247, 258, 262, 270, 273, 280, 298, 299, 313, 316, 325, 350, 355, 367, 370, 378, 383, 386, 407, 408, 410, 414, 415, 418, 429, 460, 483, 515, 518, 521, 542, 550, 554, 560, 576, 583, 585, 587, 588, 590, 592, 598, 610, 613, 620, 650, 661, 663, 669, 674, 688, 699, 704, 71

In [ ]:
{[col_names]: [funcs], ...}


In [ ]:
def slice(array, func):
    ...

def optigroup(array, slices, jitfuncs):
    ...    

def group_result(array, groups, funcs):
    result = np.empty((len(groups), len(funcs)))
    

In [23]:
import numpy as np
import numba as nb

# This function slices the array based on the provided indices
def slice(array, indices):
    return array[indices]

# This function organizes the array slices and functions
def optigroup(array, slices, jitfuncs):
    results = []
    for indices in slices:
        group = slice(array, indices)
        group_results = [func(group) for func in jitfuncs]
        results.append(group_results)
    return results


@nb.jit(parallel=True, fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True, no_rewrites=True ,nopython=True, nogil=True)
def group_result(array, groups, agg_sum, agg_mean):
    result = np.empty((len(groups), 2))  # Assuming 2 functions: agg_sum and agg_mean
    for i in prange(len(groups)):
        group_array = array[groups[i]]
        result[i, 0] = agg_sum(group_array)  # Apply agg_sum
        result[i, 1] = agg_mean(group_array)  # Apply agg_mean
    return result
    
def agg_sum(x):
    return np.sum(x)

def agg_mean(x):
    return np.mean(x)

njit=nb.jit(parallel=True, fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True, no_rewrites=True ,nopython=True, nogil=True)
# Example usage:
df = pdo.DataFrame(np.random.randn(1000000, 4), columns=['A', 'B', 'C', 'D'])#.astype(np.float32)
df['E'] = df['A'].apply(lambda x: 1 if x <-2 else 2 if x <-1 else 3 if x<0 else 4 if x<2 else 5)
array = df[['A', 'B', 'C', 'D']].to_numpy()
groups = [group.to_numpy() for group in df.groupby('E').groups.values()]
jit_agg_sum = njit(agg_sum)
jit_agg_mean = njit(agg_mean)

%timeit results = optigroup(array, groups, [jit_agg_sum, jit_agg_mean])
%timeit parallel_results = group_result(array, groups, jit_agg_sum, jit_agg_mean)


A
['A', 'B', 'C', 'D']
E
22.5 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
array = df[['A']].to_numpy()
groups = [group.to_numpy() for group in df.groupby('E').groups.values()]
jit_agg_sum = njit(agg_sum)
jit_agg_mean = njit(agg_mean)

%timeit results = optigroup(array, groups, [jit_agg_sum, jit_agg_mean])
%timeit parallel_results = group_result(array, groups, jit_agg_sum, jit_agg_mean)

['A']
E
5.36 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.2 ms ± 632 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
dfa=df.to_pandas()
%timeit dfa.groupby('E')[['A']].aggregate(['mean', 'sum'])

11.7 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
array[tuple((list(groups[0]), tuple(0,1)))]

TypeError: tuple expected at most 1 argument, got 2

In [153]:
set(*sc if type(sc)==list else sc for sc in scalcs)

SyntaxError: iterable unpacking cannot be used in comprehension (1991996339.py, line 1)

In [185]:

@nb.jit
def jcustom_sum(array):
    return np.sum(array)

@nb.jit
def jcustom_mean(array):
    return np.mean(array)

@nb.jit
def jcustom_max(array):
    return np.max(array)

@nb.jit(nopython=True, parallel=True)
def reorder(_tmp_array, rawarray, groups):
    n = len(groups)
    farray = np.zeros(len(groups) + 1, dtype=nb.int32)
    c = 0
    for j in nb.prange(n):
        m = len(groups[j])
        for i in nb.prange(m):
            _tmp_array[c] = rawarray[groups[j][i]]
            c += 1
        farray[j + 1] = c
    return farray



@nb.jit(nopython=True, parallel=True)
def apply(_tmp_array, result_array, group_sep, funcs):
    n = len(group_sep) 
    m = len(funcs)
    c = 0
    for j in nb.prange(n):
        select = _tmp_array[group_sep[j]:group_sep[j+1]]
        for func in nb.prange(m):
            result_array[j, func] = funcs[func](select)


# Define the column-function mapping with custom functions
col_func_map = {
    ('A',): [jcustom_sum, jcustom_mean],
    ('B',): [jcustom_max],
    # ('B', 'C'): [jcustom_sum]
    # Add more mappings as needed
}

pdtype = {}
sfuncs = []
scols = []
stypes = []
for cols, func_list in col_func_map.items():
    col_idx = [df.columns.get_loc(col) for col in cols]
    dtype = df.dtypes.loc[list(cols)]
    if dtype.nunique() > 1:
        raise ValueError
    dtype=dtype.iloc[0]
    scols, sfuncs=pdtype.get(dtype, [[], []])
    for func in func_list:
        scols.append(col_idx)
        sfuncs.append(func)
    pdtype[dtype] = [scols, sfuncs]



for dtype, (scols, sfuncs) in pdtype.items():
    # Getting unique columns to avoid repetition
    unique_cols = set(col for sublist in scols for col in sublist)
    n, k = len(array), len(unique_cols)
    
    # Create a temporary array for contiguous memory
    _tmp_array = np.zeros((n, k), dtype=dtype)
    result_array = np.zeros((len(groups), len(sfuncs)))
    group_sep = reorder(_tmp_array, rawarray=array[:, list(unique_cols)], groups=groups)
    apply(result_array, _tmp_array, group_sep, tuple(sfuncs))
    # Assuming 'groups' is a list of indices for each group
    for group_indices in groups:
        # Reordering raw array data into _tmp_array
        
        # Process each function on the group array
        for func in sfuncs:
            result = func(_tmp_array)

            # Handle the result as needed
            print(result)  # Example: print or store the result

    
print(pdtype)
    


/home/remi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/numba/core/utils.py:639: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/tmp/ipykernel_216863/2312305388.py:36: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  result_array[j, func] = funcs[func](select)
/home/remi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/numba/core/utils.py:639: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(Tuple(type(CPUDispatcher(<function jcustom_sum at 0x7fe7ac1c82c0>)), type(CPUDispatcher(<function jcustom_mean at 0x7fe7ac1c85e0>)), type(CPUDispatcher(<function jcustom_max at 0x7fe7b75aea20>))), int64)
 
There are 22 candidate implementations:
  - Of which 22 did not match due to:
  Overload of function 'getitem': File: <numerous>: Line N/A.
    With argument(s): '(Tuple(type(CPUDispatcher(<function jcustom_sum at 0x7fe7ac1c82c0>)), type(CPUDispatcher(<function jcustom_mean at 0x7fe7ac1c85e0>)), type(CPUDispatcher(<function jcustom_max at 0x7fe7b75aea20>))), int64)':
   No match.

During: typing of intrinsic-call at /tmp/ipykernel_216863/2312305388.py (36)

File "../../../../tmp/ipykernel_216863/2312305388.py", line 36:
<source missing, REPL/exec in use?>


In [184]:

@nb.jit
def jcustom_sum(array):
    return np.sum(array)

@nb.jit
def jcustom_mean(array):
    return np.mean(array)

@nb.jit
def jcustom_max(array):
    return np.max(array)

@nb.jit(nopython=True, parallel=True)
def reorder(_tmp_array, rawarray, groups):
    n = len(groups)
    farray = np.zeros(len(groups) + 1, dtype=nb.int32)
    c = 0
    for j in nb.prange(n):
        m = len(groups[j])
        for i in nb.prange(m):
            _tmp_array[c] = rawarray[groups[j][i]]
            c += 1
        farray[j + 1] = c
    return farray



# Dictionary of JIT-compiled functions
func_dict = {
    'sum': jcustom_sum,
    'mean': jcustom_mean,
    'max': jcustom_max
}

@nb.jit(nopython=True, parallel=True)
def apply(_tmp_array, result_array, group_sep, func_keys):
    n = len(group_sep) - 1
    m = len(func_keys)
    for j in nb.prange(n):
        select = _tmp_array[group_sep[j]:group_sep[j+1]]
        for k in nb.prange(m):
            func_key = func_keys[k]
            result_array[j, k] = func_dict[func_key](select)

pdtype = {}
sfuncs = []
scols = []
for cols, func_list in col_func_map.items():
    # Your existing code to get col_idx and dtype
    scols, sfuncs = pdtype.get(dtype, [[], []])
    for func in func_list:
        scols.append(col_idx)
        sfuncs.append(func.__name__)  # Append the function name as a string
    pdtype[dtype] = [scols, sfuncs]

for dtype, (scols, sfuncs) in pdtype.items():
    # Getting unique columns to avoid repetition
    unique_cols = set(col for sublist in scols for col in sublist)
    n, k = len(array), len(unique_cols)
    
    # Create a temporary array for contiguous memory
    _tmp_array = np.zeros((n, k), dtype=dtype)
    result_array = np.zeros((len(groups), len(sfuncs)))
    group_sep = reorder(_tmp_array, rawarray=array[:, list(unique_cols)], groups=groups)
    apply(_tmp_array, result_array, group_sep, sfuncs)
    # Your existing result handling code


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'func_dict': Cannot determine Numba type of <class 'dict'>

File "../../../../tmp/ipykernel_216863/1246074749.py", line 43:
<source missing, REPL/exec in use?>


In [84]:
import numpy as np
import pandas as pd
import numba as nb

# Helper function to slice the array based on provided indices
def slice(array, indices):
    return array[indices]

@nb.jit(nb.types.float32(nb.types.float32[:]), nopython=True)
def jcustom_sum(array):
    return np.sum(array)

@nb.jit(nb.types.float32(nb.types.float32[:]), nopython=True)
def jcustom_mean(array):
    return np.mean(array)

@nb.jit(nb.types.float32(nb.types.float32[:]), nopython=True)
def jcustom_max(array):
    return np.max(array)

col_func_map = {
    'A': [jcustom_sum, jcustom_mean],
    'B': [jcustom_max],
    ('A', 'B'): ['max']
    # Add more mappings as needed
}

# Define a single aggregation function with a flag
@nb.jit(nb.types.float32(nb.types.float32[:], nb.types.unicode_type), nopython=True)
def aggregate(array, operation):
    if operation == 0:
        return jcustom_sum(array)
    elif operation == 1:
        return jcustom_mean(array)
    elif operation == 2:
        return jcustom_max(array)
    # Add more operations as needed
    else:
        raise ValueError("Unsupported operation")

# Modified group_result to handle multiple columns and functions
@nb.njit(parallel=True)
def group_result(array, groups, column_indices, funcs):
    num_groups = len(groups)
    num_funcs = len(funcs)
    result = np.empty((num_groups, len(column_indices) * num_funcs)).astype(np.float32)

    for j, col_idx in enumerate(column_indices):
        trunc_col = array[:, col_idx]
        for k, func in enumerate(funcs[j]):
            for i, gidx in enumerate(num_groups):
                
                    result_idx = j * num_funcs + k
                    result[i, result_idx] = aggregate(trunc_col[gidx], func)

    return result

# Example usage:
df = pd.DataFrame(np.random.randn(100000, 4), columns=['A', 'B', 'C', 'D']).astype(np.float32)
df['E'] = df['A'].apply(lambda x: 1 if x < -2 else 2 if x < -1 else 3 if x < 0 else 4 if x < 2 else 5)
array = df.to_numpy()

# Group indices
groups = [group.to_numpy() for group in df.groupby('E').groups.values()]

# def custom_sum(x):
#     return np.sum(x)

# def custom_mean(x):
#     return np.mean(x)

# def custom_max(x):
#     return np.max(x)

# Define custom aggregation functions



# Define the column-function mapping with custom functions
col_func_map = {
    'A': [jcustom_sum, jcustom_mean],
    'B': [jcustom_max]
    # Add more mappings as needed
}


# Define the column-function mapping with custom functions
col_func_map = {
    'A': ['max', 'mean'],
    'B': ['sum']
    # Add more mappings as needed
}

# Prepare column indices and functions for JIT
column_indices = []
funcs = []

for col, func_list in col_func_map.items():
    col_idx = np.int8(df.columns.get_loc(col))
    column_indices.append(col_idx)
    # jit_funcs = [nb.jit(f) for f in func_list]
    funcs.append(func_list)

# Execute the computation
# result = group_result(array, groups, column_indices, funcs)
array = array.astype(np.float32)
# Timing the operation
result = group_result(array, groups, column_indices, funcs)


A
E


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<class 'enumerate'>) found for signature:
 
 >>> enumerate(int64)
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload of function 'enumerate': File: numba/core/typing/builtins.py: Line 1042.
    With argument(s): '(int64)':
   No match.

During: resolving callee type: Function(<class 'enumerate'>)
During: typing of call at /tmp/ipykernel_216863/2214971155.py (51)


File "../../../../tmp/ipykernel_216863/2214971155.py", line 51:
<source missing, REPL/exec in use?>


In [77]:
column_indices

[0, 1]

In [ ]:
def agg_sum(x):
    return np.sum(x)


df.apply(agg_sum, axis=0)

In [ ]:
df.apply(lambda x: np.sum(x))

In [ ]:
df.apply(lambda x: np.sum(x))

In [ ]:
df.groupby('A').apply(agg_sum)

In [ ]:
df = pdo.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D'])#.astype(np.float32)
type(df)

In [ ]:
type(df[['A', 'B']])

In [ ]:
df.dtypes

In [ ]:
print(type(df), type(df[['A']]))
type(df.apply(np.sum))

In [ ]:
def qqq(x):
    return np.sum(x)

def zzz(x):
    return x["A"]

df.apply(zzz, axis=1)

#df[['A', 'B']].apply(apb, axis=1)


In [ ]:
    arr_slice = df[['Client', 'Month']].values
    lidx = np.ravel_multi_index(arr_slice.T,arr_slice.max(0)+1)
    unq,unqtags,counts = np.unique(lidx,return_inverse=True,return_counts=True)
    df["Nbcontrats"] = counts[unqtags]

In [ ]:
df=pa,das

In [ ]:
    arr_slice = df[['Client', 'Month']].values
    lidx = np.ravel_multi_index(arr_slice.T,arr_slice.max(0)+1)
    unq,unqtags,counts = np.unique(lidx,return_inverse=True,return_counts=True)
    df["Nbcontrats"] = counts[unqtags]